### Проставить в дополнительном столбце статусы для клиентов, прекративших работу (lost) с указанием года и месяца потери (если клиент не совершает отправки и не приносит деньги 4 месяца)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [73]:
dataset = pd.read_csv('data.csv')
dataset['Номер'] = dataset['Месяц']
# Переименуем знечения в строках с месяцами
dataset['Номер'] = dataset['Номер'].replace({'Апрель': '4', 'Август': "8", 'Декабрь': '12', 'Июль': '7',
                                  'Июнь': '6', 'Май': '5', 'Март': '3', 'Ноябрь': '11', 
                                  'Октябрь': '10', 'Сентябрь': '9', 'Февраль': '2', 'Январь': '1'})
dataset["Дата"] = '01' + "-" + dataset['Номер'].astype(str) +"-"+ dataset['Год'].astype(str)
# преобразовать столбец даты в формат даты
dataset['Дата'] = pd.to_datetime(dataset['Дата'], dayfirst=True)
df_merged = dataset.groupby('Название клиента')['Дата'].agg(['max'])
# df_merged = df.merge(dat, how='inner', left_on = 'Название клиента', right_index = True)
df_merged['data_otsheta'] = pd.to_datetime('2018-12-31')
df_merged['delta'] = (df_merged['data_otsheta']-df_merged['max'])/ np.timedelta64(1, 'M')
df_merged['lost2'] = df_merged["delta"].apply(lambda x: 'not' if x > 4 else "Работаем")
df_merged["lost"] = df_merged['max'].dt.to_period("M")
df_merged.loc[df_merged['lost2'] == 'Работаем', 'lost'] = 0
df_merged = df_merged.reset_index()
df_merged[:5]

,Название клиента,max,data_otsheta,delta,lost2,lost
0,Клиент 1,2018-11-01,2018-12-31,1.971293,Работаем,0
1,Клиент 10,2018-08-01,2018-12-31,4.993942,not,2018-08
2,Клиент 100,2018-11-01,2018-12-31,1.971293,Работаем,0
3,Клиент 101,2018-08-01,2018-12-31,4.993942,not,2018-08
4,Клиент 102,2018-10-01,2018-12-31,2.989794,Работаем,0
